<a href="https://colab.research.google.com/github/varunsawhney8/Projects/blob/main/sample21Aug.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [224]:
import pandas as pd
import numpy as np


path='/content/Reliance_Final_Ndtv_data4.xlsx'
data=pd.read_excel(path)


data=data[::-1].reset_index()
data.drop(columns=['index'],axis=True,inplace=True)

path='/content/Reliance_Nse_data2.csv'
data1=pd.read_csv(path)


data.drop(columns=['Media','Article','Keywords'],inplace=True,axis=1)

In [218]:
data.columns

Index(['Date', 'Title'], dtype='object')

In [225]:
!pip install transformers
from transformers import pipeline
classifier = pipeline('sentiment-analysis', model="ProsusAI/finbert")
list3=[]

for i in range(len(data['Title'])):
    a=(classifier(data['Title'][i]))
    a=a[0]
    a=list(a.values())[0]
    label_f={'positive':1,'negative':2,'neutral':0}
    list3.append(label_f[a])

data['finbert']=list3

del list3

In [227]:
data.head()

,Date,Title,finbert
0,2020-07-16,Reliance's Stake Sale Talks With Saudi Aramco ...,2
1,2020-07-17,Reliance To Shut Crude Refining Unit For Maint...,2
2,2020-07-20,Reliance-Google Smartphone Deal in India Threa...,2
3,2020-07-21,"Netflix, Amazon, Facebook Stocks Draw In Amate...",1
4,2020-07-22,Reliance Industries Postpones June Quarter Res...,2


In [226]:
aggregation_functions = {'Title': 'first', 'finbert': 'max'}
data = data.groupby(data['Date']).aggregate(aggregation_functions)
data.reset_index( inplace=True)
data['Date']=pd.to_datetime(data['Date']).dt.date


In [228]:
data['3day_sentiment'] = round(data['finbert'].ewm(span=3).mean())
data['7day_sentiment'] = round(data['finbert'].ewm(span=7).mean())

In [229]:
data.head()

,Date,Title,finbert,3day_sentiment,7day_sentiment
0,2020-07-16,Reliance's Stake Sale Talks With Saudi Aramco ...,2,2.0,2.0
1,2020-07-17,Reliance To Shut Crude Refining Unit For Maint...,2,2.0,2.0
2,2020-07-20,Reliance-Google Smartphone Deal in India Threa...,2,2.0,2.0
3,2020-07-21,"Netflix, Amazon, Facebook Stocks Draw In Amate...",1,1.0,2.0
4,2020-07-22,Reliance Industries Postpones June Quarter Res...,2,2.0,2.0


In [235]:
data1['Date']=pd.to_datetime(data1['Date']).dt.date
data1['perc_change1'] =data1['Close'].pct_change(periods=1)*100
data1['perc_change3'] =data1['Close'].pct_change(periods=2)*100
data1['perc_change7'] =data1['Close'].pct_change(periods=6)*100
data1.head()

,Date,Close,perc_change1,perc_change3,perc_change7
0,2020-07-16,1843.400024,NaN,NaN,NaN
1,2020-07-17,1911.699951,3.705106,NaN,NaN
2,2020-07-20,1919.900024,0.428941,4.149940,NaN
3,2020-07-21,1971.550049,2.690246,3.130727,NaN
4,2020-07-22,2004.000000,1.645911,4.380435,NaN


In [231]:
import numpy as np

data2=pd.DataFrame(columns=["Date","Label","perc_change1"])
data2['Date']=[np.nan for i in range(len(data))]
data2['perc_change1']=[np.nan for i in range(len(data2))]
data2['Label']=[np.nan for i in range(len(data2))]

label={0: 'Neutral',1:'Positive',2:'Negative'}
for i in range(len(data2)):
  data2['Date'][i]=data['Date'][i]
  data2['Label'][i]=label[data['finbert'][i]]   
data2['Date'] = pd.to_datetime(data2['Date']).dt.date

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Re

In [232]:
for i in range(len(data2)):
  for j in range(len(data1)):
    if data['Date'][i]==data1['Date'][j]:
      data2['perc_change1'][i]=data1['perc_change1'][j+1]
  count=0
  if type(data2['perc_change1'][i]) and pd.isna(data2['perc_change1'][i]):
    for j in range(len(data1)):
      if data2['Date'][i]>data1['Date'][j]:
        count+=1
    data2['perc_change1'][i]=data1['perc_change1'][count]

data2['perc_change1'][0]=np.nan

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] ==

In [238]:
data1.head(10)

,Date,Close,perc_change1,perc_change3,perc_change7
0,2020-07-16,1843.400024,NaN,NaN,NaN
1,2020-07-17,1911.699951,3.705106,NaN,NaN
2,2020-07-20,1919.900024,0.428941,4.149940,NaN
3,2020-07-21,1971.550049,2.690246,3.130727,NaN
4,2020-07-22,2004.000000,1.645911,4.380435,NaN
5,2020-07-23,2057.800049,2.684633,4.374730,NaN
6,2020-07-24,2146.149902,4.293413,7.093308,16.423450
7,2020-07-27,2156.199951,0.468283,4.781801,12.789664
8,2020-07-28,2177.699951,0.997125,1.470077,13.427779
9,2020-07-29,2096.649902,-3.721819,-2.761806,6.345254


In [237]:
data2.head(10)

,Date,Label,perc_change1
0,2020-07-16,Negative,NaN
1,2020-07-17,Negative,0.428941
2,2020-07-20,Negative,2.690246
3,2020-07-21,Positive,1.645911
4,2020-07-22,Negative,2.684633
5,2020-07-23,Positive,4.293413
6,2020-07-24,Positive,0.468283
7,2020-07-26,Negative,0.468283
8,2020-07-27,Positive,0.997125
9,2020-07-29,Negative,0.581890


In [213]:
data4=pd.DataFrame(columns=["Description","Total No. of Investment Opportunities","Profit","Loss","No Profit No Loss"])

In [214]:
profit=0
loss=0
npnl=0
for i in range(len(data2)):

  if data2['Label'][i]=='Negative' and  data2['perc_change1'][i]<0:
    profit+=1
  elif data2['Label'][i]=='Negative' and  data2['perc_change1'][i]>0:
    loss+=1
    
  if data2['Label'][i]=='Positive' and  data2['perc_change1'][i]>0:
    profit+=1
  elif data2['Label'][i]=='Positive' and  data2['perc_change1'][i]<0:
    loss+=1
  if data2['Label'][i]=='Neutral':
    npnl+=1

total= profit + loss +neutral

data4["Description"]=["Rolling 1 day-News Day Event "]
data4["Total No. of Investment Opportunities"]=len(data)-1
data4['Profit']=profit/total*100
data4['Loss']=loss/total*100
data4['No Profit No Loss']=npnl/total*100
data4.head()

,Description,Total No. of Investment Opportunities,Profit,Loss,No Profit No Loss
0,Rolling 1 day-News Day Event,174,55.797101,44.202899,26.086957


In [215]:
data.head()

,Price,Title,finbert,Date
0,NaN,Reliance's Stake Sale Talks With Saudi Aramco ...,2,2020-07-16
1,NaN,Reliance To Shut Crude Refining Unit For Maint...,2,2020-07-17
2,NaN,Reliance-Google Smartphone Deal in India Threa...,2,2020-07-20
3,NaN,"Netflix, Amazon, Facebook Stocks Draw In Amate...",1,2020-07-21
4,NaN,Reliance Industries Postpones June Quarter Res...,2,2020-07-22
